# Statistics Explained  articles and matches with OECD's Glossary

## Objective: to build a common vocabulary and construct "profiles" of the terms of this vocabulary


In [ ]:
import re
import pandas as pd
import spacy
import sys
from collections import Counter

## Run to install the language library, then comment-out
#!{sys.executable} -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000
print('Finished loading.')


In [ ]:
from datetime import datetime

def file_name(pre,ext):
    current_time = datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext

In [ ]:
import pyodbc
c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW')
cursor = c.cursor()

In [ ]:
import re
#import unicodedata as ud

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) 
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) 

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    x = re.sub(r'â.{2}',"'",x) ### !!! NEW: single quotes are read as: âXX
    
    #x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    #x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x

### Statistics explained articles

* IDs, titles from dat_link_info, with resource_information_id=1, i.e. Eurostat (see ESTAT.V1.mod_resource_information) and matching IDs from dat_article.
* Carry out data cleansing on titles.


In [ ]:
SQLCommand = """SELECT id, title 
                FROM ESTAT.V1.dat_link_info 
                WHERE resource_information_id=1 AND id IN (SELECT id FROM ESTAT.V1.dat_article) """

SE_df = pd.read_sql(SQLCommand,c)

SE_df['title'] = SE_df['title'].apply(clean)
SE_df.head(5)


### Add paragraphs titles and contents

* From dat_article_paragraph with abstract=0 (i.e. "no").
* Match article_id from dat_article_paragraph with id from dat_article.
* Carry out data cleansing on titles and paragraph contents.

In [ ]:
SQLCommand = """SELECT article_id, title, content 
                FROM ESTAT.V1.dat_article_paragraph
                WHERE abstract=0 AND article_id IN (SELECT id FROM ESTAT.V1.dat_article) """

add_content = pd.read_sql(SQLCommand,c)
add_content['title'] = add_content['title'].apply(clean)
add_content['content'] = add_content['content'].apply(clean)
add_content

### Aggregate above paragraph titles and contents  from SE articles paragraphs by article id

* Create a column _raw content_ which gathers all paragraph titles and contents in one text per article.

In [ ]:
add_content_grouped = add_content.groupby(['article_id'])[['title','content']].aggregate(lambda x: list(x))
add_content_grouped.reset_index(drop=False, inplace=True)
for i in range(len(add_content_grouped)):
    add_content_grouped.loc[i,'raw content'] = ''
    for (a,b) in zip(add_content_grouped.loc[i,'title'],add_content_grouped.loc[i,'content']):
        add_content_grouped.loc[i,'raw content'] += ' '+a + ' ' + b
add_content_grouped = add_content_grouped[['article_id','raw content']]    

add_content_grouped

### Merge raw content of SE articles with main file

* Also, add title to definition.

In [ ]:
SE_df = pd.merge(SE_df,add_content_grouped,left_on='id',right_on='article_id',how='inner')
SE_df.drop(['article_id'],axis=1,inplace=True)

SE_df['raw content'] = SE_df['title'] +'. '+SE_df['raw content']

SE_df.head(5)

### Lemmatize 'raw content'

* NLTK seems to be better than Spacy in lemmatization. Convert to lower-case before.

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.lower())
SE_df['raw content']= SE_df['raw content'].apply(lemmatize_text)
SE_df['raw content']= [' '.join(map(str, l)) for l in SE_df['raw content']]
SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.upper())
SE_df


### OECD - Glossary of Statistical Terms
https://stats.oecd.org/glossary/alpha.asp

* Scrape terms and lemmatize.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

url = "https://stats.oecd.org/glossary/alpha.asp"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
text = soup.get_text()
    
rows = soup.find_all('tr')
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
#print(cleantext)

list_rows = []
for row in rows:
    cells = row.find_all('a')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
#print(clean2)
#type(clean2)

df = pd.DataFrame(list_rows)
df.head(10)
df[0]=df[0].apply(lambda x: re.sub(r'\[',' ',x))
df[0]=df[0].apply(lambda x: re.sub(r'\]',' ',x))
df1 = df[0].str.split(',', expand=True)
df_t = df1.T
df_t=df_t[[22]]
df_t = df_t.rename(columns={22: 'term'})
nan_value = float("NaN")

df_t.replace(" ", nan_value, inplace=True)

df_t.dropna(subset = ["term"], inplace=True)
df_t.replace(" ", nan_value, inplace=True)
df_t.insert(0, 'id', range(len(df_t)))
df_t.reset_index(inplace=True)
df_t.drop(columns=['index'],inplace=True)
df_t.head()

df_t['lemmatized_term']= df_t['term'].apply(lambda x: x.lower())
df_t['lemmatized_term']= df_t['lemmatized_term'].apply(lemmatize_text)
df_t['lemmatized_term']= [' '.join(map(str, l)) for l in df_t['lemmatized_term']]
df_t['lemmatized_term']= df_t['lemmatized_term'].apply(lambda x: x.upper())
df_t

### Prepare Spacy's PhraseMatcher by building a custom vocabulary from OECD's Glossary (lemmatized_term)

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
terms = df_t['lemmatized_term'].values.tolist()
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

### Apply PhraseMatcher

* Collect results per SE article ('doc_id') in a dataframe res. Ignore matches with 2 words or less.
* Depending on length of match: columns '3-Phrases', '4-Phrases', '5-and-above-Phrases'. These will contain dictionaries with the matched lemmatized terms and their counts, in descending order of counts.
* Column 'Terms' has a dictionary with the corresponding **original terms** in OECD's Glossary and their counts in the matches.

In [ ]:
res = pd.DataFrame(index=range(len(SE_df)))
res['3-Phrases']=[[] for i in range(len(SE_df))]
res['4-Phrases']=[[] for i in range(len(SE_df))]
res['5-and-above-Phrases']=[[] for i in range(len(SE_df))]
res['Terms']=[dict() for i in range(len(SE_df))]
docs=nlp.pipe(SE_df['raw content'])
for (i,doc) in enumerate(docs):
    print(i)
    for sent in doc.sents:
        matches = matcher(sent)
        for match_id, start, end in matches:
            span = doc[start:end]
            n_words = len(span.text.split(' '))
            if n_words >= 3:
                doc_id = SE_df.loc[i,'id']
                idx = df_t.index[df_t['lemmatized_term'].str.contains(span.text,regex=False)].tolist()
                print(i,SE_df.loc[i,'title'],len(sent.text),'>',n_words,span.text,idx)
                res.loc[i,'doc_id']=doc_id
                for elem in df_t.loc[idx,'term'].values.tolist():
                    if elem in res.loc[i,'Terms'].keys():
                        res.loc[i,'Terms'][elem] +=1
                    else:
                        res.loc[i,'Terms'][elem] =1
                #res.loc[i,'Terms'].append(concepts_df.loc[idx,'term'].values.tolist())
                if n_words == 3:
                    res.loc[i,'3-Phrases'].append(span.text)
                elif n_words == 4:
                    res.loc[i,'4-Phrases'].append(span.text)
                else:
                    res.loc[i,'5-and-above-Phrases'].append(span.text)
                    
                    


In [ ]:
res['3-Phrases']= res['3-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['4-Phrases']= res['4-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['5-and-above-Phrases']= res['5-and-above-Phrases'].apply(lambda x: dict(Counter(x).most_common()))



In [ ]:
res=pd.merge(SE_df[['id','title']],res,left_on='id',right_on='doc_id',how='left')
res.drop(columns=['doc_id'],inplace=True)
res

In [ ]:
outfile = file_name('Phrase_Matcher_SE_OECD','xlsx')
res.to_excel(outfile)
